# Accessor: Data selection using pyfesom2 xarray-accesor.

## Introduction
A key feature of Pyfesom2 is to facilitate spatial selections on its unstructured triangular grid, that is currently not supported by commonly used python libraries. Xarray provides a powerful label based selection on datasets and variables (data array). A similar interface is provided (with additional convinient features) for FESOM data through pyfesom2 accessor.


Xarray's sel method takes dimension names of a dataset or datarray as arguments for selection. For rectiliner grids this provides easy interface to select arbitary points and bounding box in latitudes and longitudes. In case of FESOM grid latitudes, longitudes are not part of its dimensions, they are coordinates of dimension nod2 (that's because lat and lon are not orthogonal to each other), that means the convinent spatial selection in xarray using lat and lon as indexers is not possible for FESOM data using xarray's sel(lat=..., lon=...) method. The .select(lat=..., lon=...) method of pyfesom2 accessor makes it possible to do such selections on FESOM data using lat and lon as indexer arguments. It further extends capabilities to select arbitarary shape polygons using argument region. 

Because of these additional arguments and to minimize confusion with xarray's sel method that may still be used with FESOM data for other purposes, pyfesom2 accessor's selection method is named select.

## Load tutorial dataset
Load a tutorial dataset, for loading other datasets see [datasets](datasets.ipynb) example notebook. 

In [1]:
from pyfesom2.datasets import tutorial_dataset
pi_grid = tutorial_dataset.load() # A remote dataset
pi_grid

<xarray.Dataset>
Dimensions:  (elem: 5839, nelem: 5727, nod2: 3140, nz: 48, nz1: 47, three: 3, time: 2)
Coordinates:
    lat      (nod2) float64 dask.array<chunksize=(3140,), meta=np.ndarray>
    lon      (nod2) float64 dask.array<chunksize=(3140,), meta=np.ndarray>
  * time     (time) datetime64[ns] 1948-12-30T23:45:00 1949-12-31T23:45:00
  * nz1      (nz1) float64 -2.5 -7.5 -15.0 ... -5.525e+03 -5.825e+03 -6.125e+03
  * nz       (nz) float64 0.0 -5.0 -10.0 -20.0 ... -5.65e+03 -6e+03 -6.25e+03
    faces    (nelem, three) uint32 dask.array<chunksize=(5727, 3), meta=np.ndarray>
Dimensions without coordinates: elem, nelem, nod2, three
Data variables:
    a_ice    (time, nod2) float32 dask.array<chunksize=(1, 3140), meta=np.ndarray>
    m_ice    (time, nod2) float32 dask.array<chunksize=(2, 3140), meta=np.ndarray>
    temp     (time, nod2, nz1) float32 dask.array<chunksize=(1, 3140, 47), meta=np.ndarray>
    u        (time, elem, nz1) float32 dask.array<chunksize=(1, 5839, 47), meta=np.ndarray>
    v        (time, elem, nz1) float32 dask.array<chunksize=(1, 5839, 47), meta=np.ndarray>
    w        (time, nod2, nz) float32 dask.array<chunksize=(1, 3140, 48), meta=np.ndarray>
Attributes:
    Dataset URL:   https://swiftbrowser.dkrz.de/public/dkrz_035d8f6ff058403bb...

## Region selection

Bounding box selection can be done by passing a 4 tuple argument to region: region = (minlon, minlat, maxlon, maxlat). Shapely's Polygon can also be used  for selecting a arbitary polygon. The region selection returns reindexed faces corresponding to selected points. This subsetted data may be saved to disk using regular xarray saving methods such as subset_dataset.to_netcdf(...).

In [2]:
pi_grid.pyfesom2.select(region=(-20, 60, 50, 80))

<xarray.Dataset>
Dimensions:  (elem: 5839, nelem: 563, nod2: 320, nz: 48, nz1: 47, three: 3, time: 2)
Coordinates:
    lat      (nod2) float64 dask.array<chunksize=(320,), meta=np.ndarray>
    lon      (nod2) float64 dask.array<chunksize=(320,), meta=np.ndarray>
  * time     (time) datetime64[ns] 1948-12-30T23:45:00 1949-12-31T23:45:00
  * nz1      (nz1) float64 -2.5 -7.5 -15.0 ... -5.525e+03 -5.825e+03 -6.125e+03
  * nz       (nz) float64 0.0 -5.0 -10.0 -20.0 ... -5.65e+03 -6e+03 -6.25e+03
    faces    (nelem, three) int64 1 0 6 2 14 4 2 ... 308 318 311 313 319 312 317
Dimensions without coordinates: elem, nelem, nod2, three
Data variables:
    a_ice    (time, nod2) float32 dask.array<chunksize=(1, 320), meta=np.ndarray>
    m_ice    (time, nod2) float32 dask.array<chunksize=(2, 320), meta=np.ndarray>
    temp     (time, nod2, nz1) float32 dask.array<chunksize=(1, 320, 47), meta=np.ndarray>
    u        (time, elem, nz1) float32 dask.array<chunksize=(1, 5839, 47), meta=np.ndarray>
    v        (time, elem, nz1) float32 dask.array<chunksize=(1, 5839, 47), meta=np.ndarray>
    w        (time, nod2, nz) float32 dask.array<chunksize=(1, 320, 48), meta=np.ndarray>
Attributes:
    Dataset URL:   https://swiftbrowser.dkrz.de/public/dkrz_035d8f6ff058403bb...

In [3]:
from shapely.geometry import Polygon
polygon_region = Polygon([(-70, 30), (-10, 0), (-10, 60)])  # a triangle in atlantic
pi_grid.pyfesom2.select(region=polygon_region)

<xarray.Dataset>
Dimensions:  (elem: 5839, nelem: 207, nod2: 129, nz: 48, nz1: 47, three: 3, time: 2)
Coordinates:
    lat      (nod2) float64 dask.array<chunksize=(129,), meta=np.ndarray>
    lon      (nod2) float64 dask.array<chunksize=(129,), meta=np.ndarray>
  * time     (time) datetime64[ns] 1948-12-30T23:45:00 1949-12-31T23:45:00
  * nz1      (nz1) float64 -2.5 -7.5 -15.0 ... -5.525e+03 -5.825e+03 -6.125e+03
  * nz       (nz) float64 0.0 -5.0 -10.0 -20.0 ... -5.65e+03 -6e+03 -6.25e+03
    faces    (nelem, three) int64 0 1 4 2 7 6 3 ... 127 126 125 121 127 128 126
Dimensions without coordinates: elem, nelem, nod2, three
Data variables:
    a_ice    (time, nod2) float32 dask.array<chunksize=(1, 129), meta=np.ndarray>
    m_ice    (time, nod2) float32 dask.array<chunksize=(2, 129), meta=np.ndarray>
    temp     (time, nod2, nz1) float32 dask.array<chunksize=(1, 129, 47), meta=np.ndarray>
    u        (time, elem, nz1) float32 dask.array<chunksize=(1, 5839, 47), meta=np.ndarray>
    v        (time, elem, nz1) float32 dask.array<chunksize=(1, 5839, 47), meta=np.ndarray>
    w        (time, nod2, nz) float32 dask.array<chunksize=(1, 129, 48), meta=np.ndarray>
Attributes:
    Dataset URL:   https://swiftbrowser.dkrz.de/public/dkrz_035d8f6ff058403bb...

## Point selection

When lat and lon are provided as indexer arguments (and of same size) to select a point selection is made.

In [4]:
import numpy as np
sel_lats = np.linspace(-90,90,10) 
sel_lons = np.linspace(-180,180,10)
pi_grid.pyfesom2.select(lat= sel_lats, lon= sel_lons)

<xarray.Dataset>
Dimensions:  (elem: 5839, nod2: 10, nz: 48, nz1: 47, time: 2)
Coordinates:
    lat      (nod2) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    lon      (nod2) float64 dask.array<chunksize=(10,), meta=np.ndarray>
  * time     (time) datetime64[ns] 1948-12-30T23:45:00 1949-12-31T23:45:00
  * nz1      (nz1) float64 -2.5 -7.5 -15.0 ... -5.525e+03 -5.825e+03 -6.125e+03
  * nz       (nz) float64 0.0 -5.0 -10.0 -20.0 ... -5.65e+03 -6e+03 -6.25e+03
Dimensions without coordinates: elem, nod2
Data variables:
    a_ice    (time, nod2) float32 dask.array<chunksize=(1, 10), meta=np.ndarray>
    m_ice    (time, nod2) float32 dask.array<chunksize=(2, 10), meta=np.ndarray>
    temp     (time, nod2, nz1) float32 dask.array<chunksize=(1, 10, 47), meta=np.ndarray>
    u        (time, elem, nz1) float32 dask.array<chunksize=(1, 5839, 47), meta=np.ndarray>
    v        (time, elem, nz1) float32 dask.array<chunksize=(1, 5839, 47), meta=np.ndarray>
    w        (time, nod2, nz) float32 dask.array<chunksize=(1, 10, 48), meta=np.ndarray>
Attributes:
    Dataset URL:   https://swiftbrowser.dkrz.de/public/dkrz_035d8f6ff058403bb...

Additionally other dimensions can also be passed as indexers similar to Xarray's sel method, in that case an orthogonal selection is made. 

In [5]:
pi_grid.pyfesom2.select(lat= sel_lats, lon= sel_lons, nz1=slice(-10,-30)) 

<xarray.Dataset>
Dimensions:  (elem: 5839, nod2: 10, nz: 48, nz1: 2, time: 2)
Coordinates:
    lat      (nod2) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    lon      (nod2) float64 dask.array<chunksize=(10,), meta=np.ndarray>
  * time     (time) datetime64[ns] 1948-12-30T23:45:00 1949-12-31T23:45:00
  * nz1      (nz1) float64 -15.0 -25.0
  * nz       (nz) float64 0.0 -5.0 -10.0 -20.0 ... -5.65e+03 -6e+03 -6.25e+03
Dimensions without coordinates: elem, nod2
Data variables:
    a_ice    (time, nod2) float32 dask.array<chunksize=(1, 10), meta=np.ndarray>
    m_ice    (time, nod2) float32 dask.array<chunksize=(2, 10), meta=np.ndarray>
    temp     (time, nod2, nz1) float32 dask.array<chunksize=(1, 10, 2), meta=np.ndarray>
    u        (time, elem, nz1) float32 dask.array<chunksize=(1, 5839, 2), meta=np.ndarray>
    v        (time, elem, nz1) float32 dask.array<chunksize=(1, 5839, 2), meta=np.ndarray>
    w        (time, nod2, nz) float32 dask.array<chunksize=(1, 10, 48), meta=np.ndarray>
Attributes:
    Dataset URL:   https://swiftbrowser.dkrz.de/public/dkrz_035d8f6ff058403bb...

For convinience, select method also takes path argument that can be a shapely's LineString achieving same selection but also returns distance along path as an additional coordinate.

## Transect selection

To select a transect or trajectory in more dimensions then lat and lon, for instance lat,lon with changing depth or time, xarray's advanced indexing may be used by passing dataarrays with a common dimension as indexers in select method. The select_points method provides a more simpler and convinient interface to such selections by passing values directy.  

In [6]:
import pandas as pd
# Define selection values in time, depth, lat, lon
sel_times = pd.date_range('1949-01-01', '1949-12-31', periods=5)
sel_levels = [0, -10, -5, -30, -15]
sel_lats = np.linspace(-90,90,5) 
sel_lons = np.linspace(-180,180,5)

In [7]:
pi_grid.pyfesom2.select_points(lon=sel_lons, lat=sel_lats, nz1=sel_levels, time=sel_times)

<xarray.Dataset>
Dimensions:   (elem: 5839, nod2: 5, nz: 48)
Coordinates:
    lat       (nod2) float64 dask.array<chunksize=(5,), meta=np.ndarray>
    lon       (nod2) float64 dask.array<chunksize=(5,), meta=np.ndarray>
    time      (nod2) datetime64[ns] 1948-12-30T23:45:00 ... 1949-12-31T23:45:00
    nz1       (nod2) float64 -2.5 -7.5 -2.5 -25.0 -15.0
  * nz        (nz) float64 0.0 -5.0 -10.0 -20.0 ... -5.65e+03 -6e+03 -6.25e+03
    distance  (nod2) float64 0.0 5.017e+03 1.503e+04 2.504e+04 3.005e+04
Dimensions without coordinates: elem, nod2
Data variables:
    a_ice     (nod2) float32 dask.array<chunksize=(5,), meta=np.ndarray>
    m_ice     (nod2) float32 dask.array<chunksize=(5,), meta=np.ndarray>
    temp      (nod2) float32 dask.array<chunksize=(5,), meta=np.ndarray>
    u         (nod2, elem) float32 dask.array<chunksize=(5, 5839), meta=np.ndarray>
    v         (nod2, elem) float32 dask.array<chunksize=(5, 5839), meta=np.ndarray>
    w         (nod2, nz) float32 dask.array<chunksize=(5, 48), meta=np.ndarray>
Attributes:
    Dataset URL:   https://swiftbrowser.dkrz.de/public/dkrz_035d8f6ff058403bb...

See accessor_plotting.ipynb example for opinionated ploting such transect.

## Selection on variables

In above examples selections were performed on entire dataset, the pyfesom2 accessor can also be used on individual data variables (dataarrays). Here select_points is used on variable "temp"

In [8]:
pi_grid.pyfesom2.temp.select_points(lon=sel_lons, lat=sel_lats, nz1=sel_levels)

<xarray.DataArray 'temp' (time: 2, nod2: 5)>
dask.array<transpose, shape=(2, 5), dtype=float32, chunksize=(1, 5), chunktype=numpy.ndarray>
Coordinates:
    lat       (nod2) float64 dask.array<chunksize=(5,), meta=np.ndarray>
    lon       (nod2) float64 dask.array<chunksize=(5,), meta=np.ndarray>
  * time      (time) datetime64[ns] 1948-12-30T23:45:00 1949-12-31T23:45:00
    nz1       (nod2) float64 -2.5 -7.5 -2.5 -25.0 -15.0
    distance  (nod2) float64 0.0 5.017e+03 1.503e+04 2.504e+04 3.005e+04
Dimensions without coordinates: nod2
Attributes:
    description:  temperature
    units:        C